# 実行環境の作成

## Google Cloud Storage

下記コードでGCPに接続

In [1]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  97576      0 --:--:-- --:--:-- --:--:-- 97576
OK
Hit:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu 

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [3]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/14 02:47:16.833769 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/14 02:47:16.861039 Opening GCS connection...
2022/09/14 02:47:17.270323 Mounting file system "statistics-hyogo"...
2022/09/14 02:47:17.272883 File system has been successfully mounted.


## Google Secret Maneger

ライブラリをインストール

In [4]:
pip install google-cloud-secret-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


シークレットを呼び出す関数

In [5]:
from google.cloud import secretmanager

def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

ESTAT_APPIDを取得する

In [6]:
PROJECT_ID = 'primal-buttress-342908'
ESTAT_APPID = access_secret(PROJECT_ID,'ESTAT_APPID' )
# print(ESTAT_APPID)

## pythonのライブラリ

In [7]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

統計表を指定してメタ情報を取得する

[API仕様](https://www.e-stat.go.jp/api/api-info/e-stat-manual3-0#api_2_2)

'statsDataId'に統計表コードを指定

In [8]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [9]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
# print(meta)

## カテゴリコード一覧

カテゴリコード'cat01'のリストを作成する関数

In [ ]:
def get_estat_catefory_codes(statsDataId):
  meta= get_estat_meta(statsDataId)
 
  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # カテゴリ一覧
  categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # カテゴリコード一覧
  catagory_codes = [d.get('@code') for d in categories]

  return catagory_codes

category_codes = get_estat_catefory_codes(statsDataId)
# print(category_codes)
# print(len(category_codes))

## 地域コード一覧

地域コード'area'のリスト一覧を作成する関数

全国'00000'は除外している

In [11]:
def get_estat_area_codes(statsDataId):
  # estat-APIのメタ情報を取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # 地域一覧
  areas = next((d for d in CLASS_OBJ if d['@id'] == 'area'), None)['CLASS']

  # 地域コード一覧（全国'00000'は削除）
  area_codes = [d.get('@code') for d in areas]
  area_codes.remove('00000')

  return area_codes

area_codes = get_estat_area_codes(statsDataId)
print(area_codes)

['01000', '02000', '03000', '04000', '05000', '06000', '07000', '08000', '09000', '10000', '11000', '12000', '13000', '14000', '15000', '16000', '17000', '18000', '19000', '20000', '21000', '22000', '23000', '24000', '25000', '26000', '27000', '28000', '29000', '30000', '31000', '32000', '33000', '34000', '35000', '36000', '37000', '38000', '39000', '40000', '41000', '42000', '43000', '44000', '45000', '46000', '47000']


# estat-APIのデータを取得

まずは、総人口のデータを取得してみる

cat01に'A1101'を指定
areaは全都道府県を指定

リストを指定する場合はカンマ区切りの文字列に変換する

## estat-APIのデータ取得

In [12]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1101'
params['cdArea']=",".join(area_codes)

# print (params)

estat-APIのデータを取得する関数を作成

In [45]:
def get_estat_data(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

res = get_estat_data(params)
# print(res)

## pandas DataFrameに変換

estat-APIのレスポンスからVALUEを抜き出し、DataFrameに変換する

In [14]:
res = get_estat_data(params)

# VALUE
VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df_value = pd.json_normalize(VALUE)
# print(df_value)

カテゴリ情報は'CLASS_OBJ'に格納されている。

これもDataFrameに変換

In [15]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

df_categories = pd.json_normalize(categories)
# print(df_categories)

'VALUE'と'CLASS_OBJ'をDataFrameで外部結合する関数を作成

In [50]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # VALUE
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  df_value = pd.json_normalize(VALUE)

  for d in CLASS_OBJ :
    # キー名を取得 @time,@area,,,
    key_name = '@{}'.format(d['@id'])

    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_value = pd.merge(df_value, df_class, on=key_name, how='outer')
  
  return df_value

df_org = conv_estat_response_to_dataframe(res)
df_org

,@tab,@cat01,@area,@time,@unit,$,@tab_name,@cat01_name,@area_name,@time_name
0,00001,A1101,01000,1975100000,人,5338206,観測値,A1101_総人口,北海道,1975年度
1,00001,A1101,02000,1975100000,人,1468646,観測値,A1101_総人口,青森県,1975年度
2,00001,A1101,03000,1975100000,人,1385563,観測値,A1101_総人口,岩手県,1975年度
3,00001,A1101,04000,1975100000,人,1955267,観測値,A1101_総人口,宮城県,1975年度
4,00001,A1101,05000,1975100000,人,1232481,観測値,A1101_総人口,秋田県,1975年度
...,...,...,...,...,...,...,...,...,...,...
2111,00001,A1101,44000,2019100000,人,1135000,観測値,A1101_総人口,大分県,2019年度
2112,00001,A1101,45000,2019100000,人,1073000,観測値,A1101_総人口,宮崎県,2019年度
2113,00001,A1101,46000,2019100000,人,1602000,観測値,A1101_総人口,鹿児島県,2019年度
2114,00001,A1101,47000,2019100000,人,1453000,観測値,A1101_総人口,沖縄県,2019年度


## DataFrameの整形

使い勝手が良くなるように、DataFrameを整形していく

必要な列だけ抽出する

In [17]:
df_1 = df_org[['@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

# print(df_1)

列名を変更する

In [18]:
columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
df_1 = df_1.rename(columns=columns)

# print(df_1)

欠損値が含まれる行を削除する

In [19]:
df_1 = df_1.dropna()

# print(df_1)

categoryNameから不要な情報（categoryCode）を削除する

In [20]:
df_1['categoryName'] = df_1.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

# print(df_1)

timeCodeを4桁文字列に置換する

In [ ]:
df_1['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

# print(df_1)

ここまでを関数にまとめる

In [ ]:
def  format_estat_dataframe(df_org):
  # 必要な列だけ抽出
  df =  df_org[['@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
  df = df.rename(columns=columns)

  # 欠損データ削除
  df = df.dropna()

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

  return df

df_fmt = format_estat_dataframe(df_org)
df_fmt

## 順位を付与する

年度ごとに順位を付与

In [ ]:
def get_estat_dataframe_withrank(df_arg):

  # 年度リストを作成
  times = df_arg['timeCode'].tolist()
  times = list(set(times))

  # 返却するDataFrameの定義
  cols = ['categoryCode', 'categoryName','timeCode','timeName','areaCode','areaName','value','unit','rank']
  df_res = pd.DataFrame(index=[], columns=cols)

  # 年度毎に順位を付与
  for time in times:

    # 単年度のdataを抽出
    df_time = df_arg.copy()[df_arg['timeCode'] == time]

    # valueを数値に変換して、'rank'を付与
    df_time = df_time.astype({'value': int})
    df_time['rank'] = df_time.rank(ascending=False)['value'].astype(int).astype(str)

    # 結合
    df_res = pd.concat([df_res, df_time])

  return df_res

df_last = get_estat_dataframe_withrank(df_fmt)
df_last

# 統計表に含まれる全てのデータをDataframeに変換する

# 新しいセクション